In [5]:
# !pip install gdown snowflake-connector-python pandas sqlalchemy snowflake-sqlalchemy-q

## step 1 - Download csv from google drive

In [ ]:
import gdown
import pandas as pd
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
from snowflake.connector.pandas_tools import write_pandas


local_filename = '2022_arid2017_to_lei_xref_csv.csv'
table_name = '2022_arid2017_to_lei_xref_csv'


# Download CSV file from Google Drive
file_id = '1CteLDDX3nALU6R1eYUTXDiG9iJ8DkZ2F'
url = f'https://drive.google.com/uc?id={file_id}'

gdown.download(url, local_filename, quiet=False)

### Step 2 - For small table - read downloaded csv in df & export to snowflake 

In [ ]:
# 1. Read CSV into pandas DataFrame
df = pd.read_csv(local_filename)

# 2. Define Snowflake connection using SQLAlchemy URL
engine = create_engine(
    URL(
        user='BRIJESH',
        password='Temp@115599',
        account='agb63276',
        warehouse='COMPUTE_WH',
        database='SFN_TRAINING',
        schema='MORTGAGE',
        role='ACCOUNTADMIN'
    )
)

# 3. Upload DataFrame to Snowflake using snowflake.connector.pandas_tools
conn = engine.connect()
success, nchunks, nrows, _ = write_pandas(
    conn=conn.connection,  
    df=df,
    table_name=table_name,
    schema='MORTGAGE',
    database='SFN_TRAINING',
    auto_create_table=True,
    overwrite=True
)

conn.close()
print(f"✅ Data uploaded to Snowflake table {table_name} successfully.")
print(f"Uploaded {nrows} rows in {nchunks} chunks.")

### Step 2 - For small & large table - export csv to snowflake 

In [ ]:
import csv
import os
import snowflake.connector

# === CONFIGURATION ===
local_file = '/home/latitude/2022_public_lar_csv.csv'
table_name = '"/home/latitude/2022_public_lar"'
database = 'SFN_TRAINING'
schema = 'MORTGAGE'

# === 1. Infer schema from CSV header (all VARCHAR) ===
with open(local_file, 'r') as f:
    reader = csv.reader(f)
    header = next(reader)

columns = [f'"{col.strip()}" VARCHAR' for col in header]
ddl = f'CREATE OR REPLACE TABLE {table_name} (\n  {", ".join(columns)}\n);'

# === 2. Connect to Snowflake ===
conn = snowflake.connector.connect(
    user='BRIJESH',
    password='Temp@115599',
    account='agb63276',
    warehouse='COMPUTE_WH',
    database=database,
    schema=schema,
    role='ACCOUNTADMIN'
)
cur = conn.cursor()

# === 3. Create table dynamically ===
cur.execute(ddl)
print(f"✅ Table {table_name} created.")

# === 4. Upload CSV to user stage (not table stage) ===
stage_path = f"@~/{os.path.basename(local_file)}"  # @~ is user stage
cur.execute(f"PUT file://{local_file} @~ AUTO_COMPRESS=TRUE")
print("✅ File uploaded to user stage.")

# === 5. Load data using COPY INTO ===
cur.execute(f"""
    COPY INTO {table_name}
    FROM {stage_path}
    FILE_FORMAT = (TYPE=CSV FIELD_OPTIONALLY_ENCLOSED_BY='"' SKIP_HEADER=1)
    ON_ERROR='CONTINUE'
""")
print(f"✅ Data loaded into table {table_name}.")

# === 6. Cleanup staged file (optional) ===
cur.execute(f"REMOVE {stage_path}")
print("✅ Stage cleaned up.")

# === 7. Close connections ===
cur.close()
conn.close()
